In [ ]:
#Clone github darknet
%cd /workspace/datn
!git clone https://github.com/AlexeyAB/darknet
!chmod -R 777 darknet/

In [ ]:
#cấu hình sử dụng GPU ở Makefile và build
%cd /workspace/datn/darknet
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!make

In [ ]:
#Tải dataset và giải nén
!pip install gdown
%cd /workspace/datn/darknet/data
!gdown --fuzzy 'https://drive.google.com/file/d/15Vxq8nILYMPxJos0YMpVpqjSV-uQrpf1/view?usp=share_link'
%cd /workspace/datn/darknet/data
!unzip data.zip > /dev/null 2>&1

In [ ]:
# Kiểm tra việc giải nén dataset
import glob2
import os

files = []
for ext in ["*.png", "*.jpeg", "*.jpg", "*.txt"]:
  image_files = glob2.glob(os.path.join("data/", ext))
  files += image_files
print("Tổng số ảnh và nhãn :",len(files))

In [ ]:
# Tải pre-trained weights trên MS-COCO
# Đối với YOLOv4 leaky tải tại đây https://avtinc-my.sharepoint.com/personal/harry_nguyen_avnet_com/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fharry%5Fnguyen%5Favnet%5Fcom%2FDocuments%2FDocuments%2Fyolov4%5Fguide%2Fxilinx%5Fyolov4%5Fpretrained%2Fyolov4%2Dleaky%5Fcoco%5Ftf%5Fbest%2Eweights&parent=%2Fpersonal%2Fharry%5Fnguyen%5Favnet%5Fcom%2FDocuments%2FDocuments%2Fyolov4%5Fguide%2Fxilinx%5Fyolov4%5Fpretrained&ga=1
%cd /workspace/datn/darknet/
!wget https://pjreddie.com/media/files/darknet53.conv.74 #YOLOv3
!gdown --fuzzy 'https://drive.google.com/file/d/18v36esoXCh-PsOKwyP2GWrpYDptDY8Zf/view?usp=sharing' #YOLOv3-tiny và YOLOv3-tiny-3L
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29 #YOLOv4-tiny và YOLOv4-tiny-3L

In [ ]:
# Sau khi sửa đổi cfg file, tiến hành huấn luyện, log được lưu trong file text yolov3.txt và đồ thị hàm mất mát và mAP lưu tại chart.png 
%cd /workspace/datn/darknet/
!./darknet detector train /workspace/datn/darknet/yolov3/yolov3.data\
                          /workspace/datn/darknet/yolov3/yolov3.cfg\
                          /workspace/datn/darknet/darknet53.conv.74\
                          -dont_show -map 2>&1 | tee yolov3.txt

In [ ]:
# Đánh giá mAP
./darknet detector map /home/tuan/darknet/final_weights_cfg/yolo.data\
                        /home/tuan/darknet/final_weights_cfg/yolov3/yolov3.cfg\
                        /home/tuan/darknet/final_weights_cfg/yolov3/yolov3_best.weights

In [ ]:
# Test ảnh
./darknet detector test /home/tuan/darknet/yolov3-tiny/yolov3-tiny.data\
                        /home/tuan/darknet/yolov3-tiny/yolov3-tiny.cfg\
                        /home/tuan/darknet/yolov3-tiny_640x480/yolov3-tiny_best.weights\
                        /home/tuan/darknet/dutuav_train1070.jpg

In [ ]:
# Demo video
./darknet detector demo /home/tuan/darknet/yolov4-tiny-3l/yolov4-tiny-3l.data\
                        /home/tuan/darknet/yolov4-tiny-3l/yolov4-tiny-3l.cfg\
                        /home/tuan/darknet/backup/yolov4-tiny-3l/yolov4-tiny-3l_best.weights\
                        /home/tuan/darknet/demo/fighter1.mp4

In [ ]:
#Demo webcam
./darknet detector demo /home/tuan/darknet/yolov4-tiny-3l/yolov4-tiny-3l.data\
                        /home/tuan/darknet/yolov4-tiny-3l/yolov4-tiny-3l.cfg\
                        /home/tuan/darknet/backup/yolov4-tiny-3l/yolov4-tiny-3l_best.weights\
                        -c 0

In [ ]:
# Video benchmark
./darknet detector demo /home/tuan/darknet/final_weights_cfg/yolo.data\
                        /home/tuan/darknet/final_weights_cfg/yolov4/yolov4.cfg\
                        /home/tuan/darknet/final_weights_cfg/yolov4/yolov4_best.weights\
                        /home/tuan/darknet/test.mp4\
                        -benchmark

In [ ]:
# Lưu kết quả
for i in data/test_set/all/*.jpg; do ./darknet detector test final_weights_cfg/yolo.data final_weights_cfg/yolov4/yolov4.cfg final_weights_cfg/yolov4/yolov4_best.weights "$i" -thresh 0.15 -iou_thresh 0.25 -dont_show; mv predictions.jpg "${i%.jpg}"_yolov4-512.jpg; done

In [ ]:
#calculate anchor
./darknet detector calc_anchors /home/tuan/darknet/final_weights_cfg/yolo.data\
                        -num_of_clusters 9 -width 640 -height 480